# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Experiment,Workspace
from azureml.core.compute import ComputeTarget,AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azure.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart_failure_prediction'

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


experiment = Experiment(ws, experiment_name)

In [ ]:
cpu_compute_name = "cpucompute"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Using existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = cpu_cluster,
                             task = "classification",
                             training_data = dataset,
                             label_column_name = "DEATH_EVENT",   
                             enable_early_stopping = True,
                             featurization = 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, best_model = remote_run.get_output()
print("Best run ID:"+ best_run.id)
print(best_run)
print(best_model)
best_run_metrics = best_run.get_matrics()
print(best_run_metrics)

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='heart-failure-prediction-automl-model',model_path='outputs/model.pkl', tags={'Method':'AutoML'}, properties={'accuracy': best_run_metrics['accuracy']})
print(model)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.webservice import Aciwebservice, Webservice
from azureml.core.model import Model, Inferenceconfig
from azureml.core import Environment
from aure_ml.core.conda_dependencies import CondaDependencies

env = Environment.from_conda_specification(name='automlv2', file_path='conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)

service_name = 'heartfailurepredictionautomlv2'

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1.8, memory_gb = 4, auth_enabled = True, enable_app_insights = True)
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)
print(service.state)

service.wait_for_deployment(show_output = True)
url = service.scoring_uri
print(url)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json

data = {"data":
        [
          {
            "age": 36,
            "anaemia": 0,
            "creatinine_phosphokinase": 200,
            "diabetes": 0,
            "ejection_fraction": 30,
            "high_blood_pressure": 0,
            "platelets": 120000,
            "serum_creatinine": 1.1,
            "serum_sodium": 135,
            "sex": 1,
            "smoking": 0,
            "time": 7
          },
          {
            "age": 39,
            "anaemia": 0,
            "creatinine_phosphokinase": 300,
            "diabetes": 0,
            "ejection_fraction": 50,
            "high_blood_pressure": 0,
            "platelets": 200000,
            "serum_creatinine": 0.9,
            "serum_sodium": 250,
            "sex": 0,
            "smoking": 0,
            "time": 1
          }
      ]
    }

input_payload = json.dumps(data)

output = service.run(input_payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()